# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [64]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import ipywidgets as widgets
import pprint
import json

# Import API key
from api_keys import g_key
gmaps.configure(g_key)
gkey = g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [65]:
#load weather data from weatherpy exercise
weather_data_df = pd.read_csv("..\Resources\weather.csv") 
weather_data_df.head()

City    Lat     Lng    Temp  Max Temp  Humidity  Cloudiness  \
0   ilulissat  69.22  -51.10  48.200    48.200        87          90   
1  dzerzhinsk  56.24   43.46  57.128    60.998        87          20   
2        hilo  19.73 -155.09  76.766    82.400        65          90   
3    el cobre  20.05  -75.95  87.800    87.800        66          20   
4    cherskiy  68.75  161.30  72.968    72.968        50           0   

   Wind Speed Country        Date  
0        4.10      GL  1595704194  
1        1.38      RU  1595704237  
2        3.10      US  1595704199  
3        4.10      CU  1595704513  
4        1.55      RU  1595704221

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [66]:
#configure map
ponta_delgada = (37.74, 25.67) #just seemed like a nice center to see the whole map in the preview window!
fig = gmaps.figure(map_type='HYBRID',center =ponta_delgada, zoom_level=2) #choose map type, center and zoom
#specify heatmay layer values, coordinates, weighted for humidity
heatmap_layer = gmaps.heatmap_layer(
    weather_data_df[["Lat","Lng"]], weights=weather_data_df["Humidity"], max_intensity = 150, point_radius = 10.0)
#add layer to map
fig.add_layer(heatmap_layer)
#display
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [67]:
#Remove places where weather is <65 or >85
bool_series = weather_data_df["Max Temp"].between(65,85,inclusive=True) #geeksforgeeks.org
ideal_weather_df = weather_data_df[bool_series]                                

In [68]:
#Remove places where humidity is >30%
ideal_weather_df = ideal_weather_df[ideal_weather_df["Humidity"] <= 30]

In [69]:
#Remove places where wind speed is >15
ideal_weather_df_df = ideal_weather_df[ideal_weather_df["Wind Speed"] <= 15]

In [70]:
#Remove places where cloudiness is >20%
ideal_weather_df = ideal_weather_df[ideal_weather_df["Cloudiness"] <= 20]
#check the values in the filter data frame
ideal_weather_df.describe()

Lat         Lng       Temp   Max Temp   Humidity  Cloudiness  \
count  11.000000   11.000000  11.000000  11.000000  11.000000   11.000000   
mean   16.321818   23.169091  75.609091  75.672909  23.636364    1.818182   
std    33.000539   59.545088   5.164119   5.136886   3.294624    5.706456   
min   -33.020000 -111.300000  66.434000  66.434000  18.000000    0.000000   
25%   -18.095000   15.750000  72.401000  72.455000  21.000000    0.000000   
50%    34.500000   43.000000  77.000000  77.000000  23.000000    0.000000   
75%    40.780000   62.250000  80.267000  80.267000  26.000000    0.000000   
max    47.500000   77.980000  81.608000  81.608000  29.000000   19.000000   

       Wind Speed          Date  
count   11.000000  1.100000e+01  
mean     4.105455  1.595704e+09  
std      2.809919  1.591926e+02  
min      0.990000  1.595704e+09  
25%      1.875000  1.595704e+09  
50%      3.880000  1.595705e+09  
75%      6.090000  1.595705e+09  
max      9.000000  1.595705e+09

In [71]:
ideal_weather_df

City    Lat     Lng    Temp  Max Temp  Humidity  \
9        sukhaya buyvola  45.15   43.00  77.000    77.000        29   
82           great falls  47.50 -111.30  73.292    73.994        23   
118          east london -33.02   27.91  71.510    71.510        27   
211              ushtobe  45.25   77.98  77.378    77.378        25   
255  santiago del estero -27.80  -64.26  69.368    69.368        21   
256         torbat-e jam  35.24   60.62  81.608    81.608        21   
270             luderitz -26.65   15.16  73.400    73.400        27   
280              malanje  -9.54   16.34  66.434    66.434        25   
335           dawlatabad  36.41   64.91  80.996    80.996        18   
337                farah  32.50   62.50  79.538    79.538        23   
395                herat  34.50   62.00  81.176    81.176        21   

     Cloudiness  Wind Speed Country        Date  
9            19        9.00      RU  1595704514  
82            1        2.10      US  1595704267  
118           0        3.88      ZA  1595704241  
211           0        4.15      KZ  1595704320  
255           0        5.86      AR  1595704552  
256           0        6.32      IR  1595704552  
270           0        2.10     NaN  1595704204  
280           0        1.18      AO  1595704556  
335           0        0.99      AF  1595704235  
337           0        1.65      AF  1595704565  
395           0        7.93      AF  1595704575

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [72]:
#Store ideal weather conditions in a new dataframe
hotel_df=ideal_weather_df.copy()

#Add a column to the dataframe
hotel_df["Hotel"]=""

#Iterate through hotel_df to populate the hotel column
for index,row in hotel_df.iterrows():

    try:
        target_coordinates = f"{row['Lat']}, {row['Lng']}" 
        target_keyword = "lodging"
        target_type = "lodging" 
        target_radius = "5000"

        params = {
            "keyword": target_keyword,
            "location": target_coordinates,
            "radius": target_radius,
            "type": target_type,
            "key": gkey
        }

        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        response = requests.get(base_url, params=params)
        data=response.json()
        hotel_df.loc[index,"Hotel"] = data["results"][0]["name"]
        
    except:
        print(f"You can't stay in {row['City']}, {row['Country']}...unless you bring a tent or you know somebody!")
        
hotel_df

You can't stay in sukhaya buyvola, RU...unless you bring a tent or you know somebody!
You can't stay in farah, AF...unless you bring a tent or you know somebody!
You can't stay in herat, AF...unless you bring a tent or you know somebody!


City    Lat     Lng    Temp  Max Temp  Humidity  \
9        sukhaya buyvola  45.15   43.00  77.000    77.000        29   
82           great falls  47.50 -111.30  73.292    73.994        23   
118          east london -33.02   27.91  71.510    71.510        27   
211              ushtobe  45.25   77.98  77.378    77.378        25   
255  santiago del estero -27.80  -64.26  69.368    69.368        21   
256         torbat-e jam  35.24   60.62  81.608    81.608        21   
270             luderitz -26.65   15.16  73.400    73.400        27   
280              malanje  -9.54   16.34  66.434    66.434        25   
335           dawlatabad  36.41   64.91  80.996    80.996        18   
337                farah  32.50   62.50  79.538    79.538        23   
395                herat  34.50   62.00  81.176    81.176        21   

     Cloudiness  Wind Speed Country        Date  \
9            19        9.00      RU  1595704514   
82            1        2.10      US  1595704267   
118           0        3.88      ZA  1595704241   
211           0        4.15      KZ  1595704320   
255           0        5.86      AR  1595704552   
256           0        6.32      IR  1595704552   
270           0        2.10     NaN  1595704204   
280           0        1.18      AO  1595704556   
335           0        0.99      AF  1595704235   
337           0        1.65      AF  1595704565   
395           0        7.93      AF  1595704575   

                                         Hotel  
9                                               
82   SpringHill Suites by Marriott Great Falls  
118                                   Stirling  
211                         Gostinitsa "Diyar"  
255                             Hotel Coventry  
256                           Pardisan Hotel 🏩  
270                        LÜDERITZ NEST HOTEL  
280                              Palanca Hotel  
335                           WASIM ASGAR KHAN  
337                                             
395

In [73]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [74]:
# Add marker layer ontop of heat map
marker_locations= hotel_df[["Lat","Lng"]]
fig = gmaps.figure(map_type='HYBRID',center =ponta_delgada, zoom_level=2)
heatmap_layer = gmaps.heatmap_layer(
    weather_data_df[["Lat","Lng"]], weights=weather_data_df["Humidity"], max_intensity = 150, point_radius = 10.0)
fig.add_layer(heatmap_layer)
markers = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))